The code presented here is not meant to be executed from a notebook. However, the notebook format offers a nice format for including, well, notes, and explanations.

- Get synchrotron meshes
- Generate segmentations from sunchrotron meshes
- Generate B-fields
- Run Monte Carlo simulations
- Compute signals

# GENERATE SEGMENTATIONS FROM SYNCHROTRON MESHES

In [7]:
! pwd

/home/siwin/code/susceptibility-and-axon-morphology/notebooks


# GENERATE B-FIELDS FROM SYNCHROTRON MESHES